注意：執行此範例需要用到大量的 RAM (必須是 Colab Pro 用戶)

In [ ]:
import tensorflow
print(tensorflow.__version__)

import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from PIL import Image

# 指定亂數種子
seed = 10
np.random.seed(seed)

# 載入資料集
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# 打亂 2 個 Numpy 陣列
def randomize(a, b):
    permutation = list(np.random.permutation(a.shape[0]))
    shuffled_a = a[permutation]
    shuffled_b = b[permutation]

    return shuffled_a, shuffled_b

x_train, y_train = randomize(x_train, y_train)
x_test, y_test = randomize(x_test, y_test)

# 取出10%訓練, 10%測試
x_train = x_train[:5000]
y_train = y_train[:5000]
x_test = x_test[:1000]
y_test = y_test[:1000]

# One-hot編碼
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 載入 ResNet50 模型
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

# 調整 x_train 的圖片尺寸
print('調整 x_train 的圖片尺寸...')
x_train_new = np.array(
  [np.asarray(Image.fromarray(x_train[i]).resize(
          (200, 200))) for i in range(0, len(x_train))])
x_train_new = x_train_new.astype('float32')

# 訓練資料的資料前處理
train_input = preprocess_input(x_train_new)

# 使用 ResNet50 模型預測訓練資料的特徵資料
print('使用 ResNet50 模型預測訓練資料的特徵資料...')
train_features = resnet_model.predict(train_input)

# 調整 x_test 的圖片尺寸
print('調整 x_test 的圖片尺寸...')
x_test_new = np.array(
  [np.asarray(Image.fromarray(x_test[i]).resize(
          (200, 200))) for i in range(0, len(x_test))])

x_test_new = x_test_new.astype('float32')

# 測試資料的資料前處理
test_input = preprocess_input(x_test_new)

# 使用 ResNet50 模型預測測試資料的特徵資料
print('"使用 ResNet50 模型預測測試資料的特徵資料...')
test_features = resnet_model.predict(test_input)

# 定義模型
model = Sequential()
model.add(GlobalAveragePooling2D(input_shape=train_features.shape[1:]))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# 編譯模型
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# 訓練模型
history = model.fit(train_features, y_train, validation_data=(test_features, y_test), epochs=15, batch_size=32, verbose=2)

# 評估模型
print('\nTesting ...')
loss, accuracy = model.evaluate(test_features, y_test, verbose=0)
print('測試資料集的準確度 = {:.2f}'.format(accuracy))

# 顯示圖表來分析模型的訓練過程
import matplotlib.pyplot as plt

# 顯示訓練和驗證損失
loss = history.history['loss']
epochs = range(1, len(loss)+1)
val_loss = history.history['val_loss']
plt.plot(epochs, loss, 'bo-', label='Training Loss')
plt.plot(epochs, val_loss, 'ro--', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# 顯示訓練和驗證準確度
acc = history.history['accuracy']
epochs = range(1, len(acc)+1)
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'bo-', label='Training Acc')
plt.plot(epochs, val_acc, 'ro--', label='Validation Acc')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

2.15.0
94765736/94765736 [==============================] - 0s 0us/step
調整 x_train 的圖片尺寸...
使用 ResNet50 模型預測訓練資料的特徵資料...
157/157 [==============================] - 17s 79ms/step
調整 x_test 的圖片尺寸...
"使用 ResNet50 模型預測測試資料的特徵資料...
32/32 [==============================] - 2s 74ms/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 10)                20490     
                                                                 
Total params: 20490 (80.04 KB)
Trainable params: 20490 (80.04 KB)
Non-trainable params: 0 (0.00 Byte)
___